<h2 align="center">Importing Libraries</h2>


In [2]:
import sqlalchemy
from urllib.parse import quote_plus
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline 
import seaborn as sns 

<h2 align="center">Import Data</h2>

In [3]:
Username = 'root'
Password = 'Kamal@123'
Host = 'localhost'
Database = 'codex'
encoded_password = quote_plus(Password)

engine = sqlalchemy.create_engine(f'mysql+mysqldb://{Username}:{encoded_password}@{Host}/{Database}')

<h2 align="center">Initial Exploration</h2>

In [4]:
Tables = pd.read_sql_query("show tables",engine)["Tables_in_codex"].to_list()
Tables

['brand_marketing',
 'dim_cities',
 'dim_repondents',
 'fact_survey_responses',
 'marketing_types']

In [5]:
pd.set_option("display.max_columns", None)
for Table in Tables:
    print(Table, "Table")
    display(pd.read_sql_query(F"SELECT * FROM {Table} limit 5", engine))

brand_marketing Table


,Marketing Sub Type,Platform,Platform Rating
0,Influencer Marketing,Athlete Celebs/Brand Ambassador,4.5
1,Social Media Marketing,Athlete & Energetic Social Media Influencers,5.0
2,Content Marketing,Athlete & Energetic Social Media Influencers,5.0
3,Event Marketing,Sunburn Festival,5.0
4,Event Marketing,Red Bull BC One India,5.0


dim_cities Table


,City_ID,City,Tier
0,CT111,Delhi,Tier 1
1,CT112,Mumbai,Tier 1
2,CT113,Bangalore,Tier 1
3,CT114,Chennai,Tier 1
4,CT115,Kolkata,Tier 2


dim_repondents Table


,Respondent_ID,Name,Age,Gender,City_ID
0,120031,Aniruddh Issac,15-18,Female,CT117
1,120032,Trisha Rout,19-30,Male,CT118
2,120033,Yuvraj Virk,15-18,Male,CT116
3,120034,Pranay Chand,31-45,Female,CT113
4,120035,Mohanlal Joshi,19-30,Female,CT120


fact_survey_responses Table


,Response_ID,Respondent_ID,Consume_frequency,Consume_time,Consume_reason,Heard_before,Brand_perception,General_perception,Tried_before,Taste_experience,Reasons_preventing_trying,Current_brands,Reasons_for_choosing_brands,Improvements_desired,Ingredients_expected,Health_concerns,Interest_in_natural_or_organic,Marketing_channels,Packaging_preference,Limited_edition_packaging,Price_range,Purchase_location,Typical_consumption_situations
0,103001,120031,2-3 times a week,To stay awake during work/study,Increased energy and focus,Yes,Neutral,Not sure,No,5,Unfamiliar with the brand,Blue Bull,Availability,Reduced sugar content,Guarana,No,Yes,TV commercials,Compact and portable cans,Yes,50-99,Supermarkets,Studying/working late
1,103002,120032,2-3 times a month,Throughout the day,To boost performance,No,Neutral,Not sure,No,5,Not interested in energy drinks,Bepsi,Availability,More natural ingredients,Caffeine,Yes,Not Sure,Print media,Compact and portable cans,No,50-99,Supermarkets,Sports/exercise
2,103003,120033,Rarely,Before exercise,Increased energy and focus,No,Neutral,Not sure,No,2,Not available locally,Bepsi,Availability,More natural ingredients,Caffeine,No,Yes,Online ads,Innovative bottle design,Not Sure,100-150,Supermarkets,Studying/working late
3,103004,120034,2-3 times a week,To stay awake during work/study,To boost performance,No,Positive,Dangerous,Yes,5,Other,Bepsi,Other,Other,Caffeine,No,Yes,Online ads,Compact and portable cans,No,Above 150,Supermarkets,Sports/exercise
4,103005,120035,Daily,To stay awake during work/study,Increased energy and focus,Yes,Neutral,Effective,Yes,5,Other,Sky 9,Other,More natural ingredients,Caffeine,Yes,Yes,Online ads,Compact and portable cans,Yes,100-150,Online retailers,Studying/working late


marketing_types Table


,Marketing Channels,Marketing Types,Marketing Sub types,Rating
0,Online ads,Digital Marketing,Influencer Marketing,5
1,Online ads,Digital Marketing,Social Media Marketing,4
2,Online ads,Digital Marketing,Content Marketing,4
3,Online ads,Digital Marketing,Search Engine Marketing,3
4,Online ads,Digital Marketing,Email Marketing,3


In [6]:
for Table in Tables:
    display(pd.read_sql_query(f"SELECT * FROM {Table}", engine).dtypes)

Marketing Sub Type     object
Platform               object
Platform Rating       float64
dtype: object

City_ID    object
City       object
Tier       object
dtype: object

Respondent_ID     int64
Name             object
Age              object
Gender           object
City_ID          object
dtype: object

Response_ID                        int64
Respondent_ID                      int64
Consume_frequency                 object
Consume_time                      object
Consume_reason                    object
Heard_before                      object
Brand_perception                  object
General_perception                object
Tried_before                      object
Taste_experience                   int64
Reasons_preventing_trying         object
Current_brands                    object
Reasons_for_choosing_brands       object
Improvements_desired              object
Ingredients_expected              object
Health_concerns                   object
Interest_in_natural_or_organic    object
Marketing_channels                object
Packaging_preference              object
Limited_edition_packaging         object
Price_range                       object
Purchase_location                 object
Typical_consumption_situations    object
dtype: object

Marketing Channels     object
Marketing  Types       object
Marketing Sub types    object
Rating                  int64
dtype: object

<h2 align="center">Analysis</h2>

### 1. Demographic Insights (examples)

- A: Who prefers energy drink more? (male/female/non-binary?)

In [7]:
pd.read_sql_query('''
                  WITH GENDER_COMPARISION AS (
                  SELECT F.CONSUME_FREQUENCY, R.GENDER, COUNT(F.RESPONDENT_ID) AS CNT,
                  DENSE_RANK() OVER(PARTITION BY F.CONSUME_FREQUENCY ORDER BY COUNT(F.RESPONDENT_ID) DESC) AS RANK_
                  FROM dim_repondents R JOIN fact_survey_responses F ON R.RESPONDENT_ID = F.RESPONDENT_ID
                  GROUP BY F.CONSUME_FREQUENCY, R.GENDER)

                  SELECT CONSUME_FREQUENCY, GENDER, RANK_ FROM GENDER_COMPARISION
                  WHERE RANK_ = 1
             ''', engine)

,CONSUME_FREQUENCY,GENDER,RANK_
0,2-3 times a month,Male,1
1,2-3 times a week,Male,1
2,Daily,Male,1
3,Once a week,Male,1
4,Rarely,Male,1


In [8]:
# MALE CONSUME ENERGY DRINK MORE THAN FEMALE IN EVERY CONSUME FREQUENCY

- B: Which age group prefers energy drinks more?

In [13]:
pd.read_sql_query(''' 
                WITH AGE_GROUP_COMPARISION AS (
                SELECT F.CONSUME_FREQUENCY, D.AGE,
                DENSE_RANK() OVER(PARTITION BY F.CONSUME_FREQUENCY ORDER BY COUNT(D.RESPONDENT_ID) DESC ) AS RANK_ 
                FROM fact_survey_responses F JOIN dim_repondents D ON F.RESPONDENT_ID = D.RESPONDENT_ID
                GROUP BY F.CONSUME_FREQUENCY, D.AGE
                )
                SELECT CONSUME_FREQUENCY, AGE FROM AGE_GROUP_COMPARISION
                WHERE RANK_ = 1

                    ''', engine)

,CONSUME_FREQUENCY,AGE
0,2-3 times a month,19-30
1,2-3 times a week,19-30
2,Daily,19-30
3,Once a week,19-30
4,Rarely,19-30


In [ ]:
# 19-30 AGE GROUP OF PEOPLE IS MORE LIKE TO HAVE ENERGY DRINK

- c: Which type of marketing reaches the most Youth (15-30)?

In [16]:
pd.read_sql_query(''' 
                  SELECT F.MARKETING_CHANNELS , COUNT(D.RESPONDENT_ID) AS CNT	
                  FROM dim_repondents D JOIN fact_survey_responses F ON D.RESPONDENT_ID = F.RESPONDENT_ID
                  WHERE D.AGE IN ( '15-18', '19-30' )
                  GROUP BY F.MARKETING_CHANNELS
                  ORDER BY CNT DESC
                  LIMIT 1
                  ''', engine)

,MARKETING_CHANNELS,CNT
0,Online ads,3373


In [ ]:
# 'ONLINE ADS' IS MOST POPULAR MARKETING CHANNEL AMONG THE YOUTH (15-30)

In [ ]:
['brand_marketing',
 'dim_cities',
 'dim_repondents',
 'fact_survey_responses',
 'marketing_types']

In [ ]:
pd.set_option("display.max_columns", None)
for Table in Tables:
    print(Table, "Table")
    display(pd.read_sql_query(F"SELECT * FROM {Table} limit 5", engine))

brand_marketing Table


,Marketing Sub Type,Platform,Platform Rating
0,Influencer Marketing,Athlete Celebs/Brand Ambassador,4.5
1,Social Media Marketing,Athlete & Energetic Social Media Influencers,5.0
2,Content Marketing,Athlete & Energetic Social Media Influencers,5.0
3,Event Marketing,Sunburn Festival,5.0
4,Event Marketing,Red Bull BC One India,5.0


dim_cities Table


,City_ID,City,Tier
0,CT111,Delhi,Tier 1
1,CT112,Mumbai,Tier 1
2,CT113,Bangalore,Tier 1
3,CT114,Chennai,Tier 1
4,CT115,Kolkata,Tier 2


dim_repondents Table


,Respondent_ID,Name,Age,Gender,City_ID
0,120031,Aniruddh Issac,15-18,Female,CT117
1,120032,Trisha Rout,19-30,Male,CT118
2,120033,Yuvraj Virk,15-18,Male,CT116
3,120034,Pranay Chand,31-45,Female,CT113
4,120035,Mohanlal Joshi,19-30,Female,CT120


fact_survey_responses Table


,Response_ID,Respondent_ID,Consume_frequency,Consume_time,Consume_reason,Heard_before,Brand_perception,General_perception,Tried_before,Taste_experience,Reasons_preventing_trying,Current_brands,Reasons_for_choosing_brands,Improvements_desired,Ingredients_expected,Health_concerns,Interest_in_natural_or_organic,Marketing_channels,Packaging_preference,Limited_edition_packaging,Price_range,Purchase_location,Typical_consumption_situations
0,103001,120031,2-3 times a week,To stay awake during work/study,Increased energy and focus,Yes,Neutral,Not sure,No,5,Unfamiliar with the brand,Blue Bull,Availability,Reduced sugar content,Guarana,No,Yes,TV commercials,Compact and portable cans,Yes,50-99,Supermarkets,Studying/working late
1,103002,120032,2-3 times a month,Throughout the day,To boost performance,No,Neutral,Not sure,No,5,Not interested in energy drinks,Bepsi,Availability,More natural ingredients,Caffeine,Yes,Not Sure,Print media,Compact and portable cans,No,50-99,Supermarkets,Sports/exercise
2,103003,120033,Rarely,Before exercise,Increased energy and focus,No,Neutral,Not sure,No,2,Not available locally,Bepsi,Availability,More natural ingredients,Caffeine,No,Yes,Online ads,Innovative bottle design,Not Sure,100-150,Supermarkets,Studying/working late
3,103004,120034,2-3 times a week,To stay awake during work/study,To boost performance,No,Positive,Dangerous,Yes,5,Other,Bepsi,Other,Other,Caffeine,No,Yes,Online ads,Compact and portable cans,No,Above 150,Supermarkets,Sports/exercise
4,103005,120035,Daily,To stay awake during work/study,Increased energy and focus,Yes,Neutral,Effective,Yes,5,Other,Sky 9,Other,More natural ingredients,Caffeine,Yes,Yes,Online ads,Compact and portable cans,Yes,100-150,Online retailers,Studying/working late


marketing_types Table


,Marketing Channels,Marketing Types,Marketing Sub types,Rating
0,Online ads,Digital Marketing,Influencer Marketing,5
1,Online ads,Digital Marketing,Social Media Marketing,4
2,Online ads,Digital Marketing,Content Marketing,4
3,Online ads,Digital Marketing,Search Engine Marketing,3
4,Online ads,Digital Marketing,Email Marketing,3
